# Introduction to PDEpy
This is a numerical library for solving PDEs with finite difference method.
## Supported PDEs
### 2D Linear (Constant/Variable Coefficient) PDEs on Regular/Irregular Domains with Dirichlet Boundary Condition
Default rate of convergence: second-order of accuracy.  
Documentations: https://github.com/Walden-Shen/pdepy/blob/master/docs/2dPDEs.ipynb

### Linear (Constant/Variable coefficient) Time-dependent PDEs
#### 1D Problems
Default algorithm: Crank-Nicolson method (fourth-order of accuracy & unconditionally stable).  
Documentations: https://github.com/Walden-Shen/pdepy/blob/master/docs/time_dependent_PDEs.ipynb

#### 2D Problems on Regular Domains
Default algorithm: the explicit method (conditionally stable).  
Documentations: https://github.com/Walden-Shen/pdepy/blob/master/docs/time_dependent_PDEs.ipynb

# Gist of This Library
- User Friendly  
Whatever type of PDE the user wants to solve, all he/she needs to do is to set the boundary condition, plug PDE into the solver, and finally call the function **solve()**. No need to memorize tons of APIs, since the solver will automatically detect the type of the PDE, whether its domain is regular, etc., and select the appropriate algorithm for you.  
     
     
- Extensible & Easy to Extend  
It's impossible that the library will meet all needs from its users, e.g., higher-order differential operators, more accurate approximation, etc. So when I was designing this library, I made it extremely easy to add new operators, or to modify the existing numerical stencil.  

# Code Structure

## src.util.diff_operators
There are two child packages __core__ and __impl__ inside it. **diff_operators.core** contains the most generic operators:
```python
class diff_operator(object):
    def __init__(self, stencil, coefficient = lambda x, y: 1, is_time_dependent = False):
        # a list of tuple whose first element is relative position of node in the stencil, 
        # second element is the coefficient in the fdm equation
        self.stencil = stencil
        # the coefficient before this operator, e.g., lambda x, y: 1/delta_x^2
        self.coefficient = coefficient
        self.is_time_dependent = is_time_dependent
    def getIrregularStencil(self, dx, tau, direction):
        raise NotImplementedError
```
and
```python
class time_dependent_operator(diff_op.diff_operator):
    def __init__(self, implicit_stencil, explicit_stencil = None, coefficient = lambda x, y: 1):
        super().__init__(implicit_stencil, coefficient, True)
        self.explicit_stencil = explicit_stencil 
        # for 2d time-dependent equation. The first argument in tuple represents x coordinate offset,
        # while the second argument represents y coordinate offset
    def get_implicit_stencil(self):
        return self.stencil
    def get_explicit_stencil(self):
        return self.explicit_stencil
```
It's intuitive that the time-dependent operator class is a subclass of usual operator class. And I make the type of coefficient be a lambda function, which **allows the coefficient before the operator to be either a function g(x, y) or a constant**.  
Then let's take a look at concrete operators in **diff_operators.impl**.
```python
class ddx(diff_op.diff_operator):
    def __init__(self, dx, coefficient = lambda x, y: 1):
        super().__init__([((-1, 0), -1/(2*dx)), ((1, 0), 1/(2*dx))], coefficient)
```
The first argument $[((-1, 0), -1/(2*dx)), ((1, 0), 1/(2*dx))]$ represents the stencil we used to approximate $\frac{du}{dx}$, which is $\frac{u(x+\Delta x, y)-u(x-\Delta x, y)}{2\Delta x}\sim \frac{du}{dx} + \mathcal{O}(\Delta x^2)$.   
From this example, we can see that $(-1, 0)$ represents $(x-\Delta x, y)$ and $-1/(2*dx)$ represents $-\frac{1}{2\Delta x}$.   
Also note that the lambda function **coefficient** means the coefficient before this operator, e.g., if we have $-sin(x)sin(y)\frac{du}{dx}$, the **coefficient** would be __lambda x, y: -math.sin(x)\*math.sin(y)__.
## src.util.diff_op_expression
This is a class for storing all the operators in a PDE. It can be seen as a list that is able to analyze what type of PDE is inputed. 
## src.util.domain_conditions
This is one of the boundary condition that users have to fill in. The arguments' names are self-explanatory.
```python
class dirichlet_bc(bc.boundary_condition):
    def __init__(self, inDomain, onBoundary, getBoundaryValue, domain, getNearestPoint = (None, None)):
        super().__init__(inDomain, onBoundary, getBoundaryValue, getNearestPoint)
        self.domain = domain # this is the class domain
```
## src.fdm